In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
from scipy import stats
import math
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import pandas as pd
import geopandas as gpd

In [2]:
def custom_dist(u, v):
    if u[2] == v[2]:
        return sys.float_info.max
    else:
        return haversine([u[0], u[1]], [v[0], v[1]])

thresholds = {'CurbRamp': 0.0075,
                      'NoCurbRamp': 0.0075,
                      'SurfaceProblem': 0.01,
                      'Obstacle': 0.01,
                      'NoSidewalk': 0.01,
                      'Crosswalk': 0.01,
                      'Signal': 0.01,
                      'Occlusion': 0.01,
                      'Other': 0.01,
                      'Problem': 0.01}

def cluster(labels, label_type, thresholds):

    # Makes a normal dist matrix for a single user, but uses special dist function for multi-user clustering that
    # prevents the same user's attributes from being clustered together.
    dist_matrix = pdist(np.array(labels[['lat', 'lng', 'user_id']].values), custom_dist)
    link = linkage(dist_matrix, method='complete')
    curr_type = labels['label_type'].values

    # Copies the labels dataframe and adds a column to it for the cluster id each label is in.
    labelsCopy = labels.copy()
    labelsCopy.loc[:,'cluster_id'] = fcluster(link, t=thresholds[label_type], criterion='distance')

    # Cuts tree so that only labels less than clust_threth kilometers apart are clustered.
    clusters = labelsCopy.groupby('cluster_id')

    # Computes the center of each cluster and assigns temporariness and severity.
    cluster_list = [] # list of tuples (label_type, cluster_num, lat, lng, severity, temporary).
    for clust_num, clust in clusters:
        #ave_pos = np.mean(clust['coords'].tolist(), axis=0) # use ave pos of clusters.
        #ave_sev = None if pd.isnull(clust['severity']).all() else int(round(np.median(clust['severity'][~np.isnan(clust['severity'])])))
        #ave_temp = None if pd.isnull(clust['temporary']).all() else bool(round(np.mean(clust['temporary'])))

        cluster_list.append((curr_type, clust_num))

    cluster_df = pd.DataFrame(cluster_list, columns=['label_type', 'cluster_id'])

    return (cluster_df, labelsCopy)


# Pick which label types should be included in clustering, and which should be included in the "Problem" type.
label_types = ['CurbRamp', 'NoSidewalk', 'Occlusion', 'SurfaceProblem', 'Obstacle', 'Other', 'NoCurbRamp', 'Crosswalk', 'Signal']
# problem_types = ['SurfaceProblem', 'Obstacle']

# These are the columns required in the POST requests for the labels and clusters, respectively.
label_cols = ['label_id', 'label_type', 'cluster_id']
cluster_cols = ['label_type', 'cluster_id', 'lat', 'lng', 'severity', 'temporary']


def cluster_label_type_at_index(label_data,i):
        clusters_for_type_i = pd.DataFrame(columns=cluster_cols)
        labels_for_type_i = pd.DataFrame(columns=label_cols)

        label_type = label_types[i]
        # if label_type == 'Problem':
        #     type_data = label_data[label_data.label_type.isin(problem_types)]
        # else:
        type_data = label_data[label_data.label_type == label_type]

        # If there are >1 labels, we can do clustering. Otherwise just copy the 1 (or 0) labels.
        if type_data.shape[0] > 1:
            (clusters_for_type_i, labels_for_type_i) = cluster(type_data, label_type, thresholds)
        elif type_data.shape[0] == 1:
            labels_for_type_i = type_data.copy()
            labels_for_type_i.loc[:,'cluster_id'] = 1 # Gives the single cluster a cluster_id of 1.
            labels_for_type_i.loc[:,'label_type'] = label_type # Gives Problem type if needed.
            clusters_for_type_i = labels_for_type_i.filter(items=cluster_cols)

        return (label_type, clusters_for_type_i, labels_for_type_i)

In [3]:
df = gpd.read_file('data/attributesWithLabels_q03.json')

In [4]:
df

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,...,label_date,label_severity,label_is_temporary,agree_count,disagree_count,notsure_count,label_tags,label_description,user_id,geometry
0,68037009,CurbRamp,19897,263712791,Atlantic,1.0,False,15,jQdLwvjVMqab7AAsMcrVQA,87.004463,...,2019-03-04 20:06:00.321,1.0,False,1,0,0,[ ],None,e4ff325c-309b-4ea6-8555-8f973e662362,POINT (-122.30888 47.59828)
1,67992285,NoCurbRamp,12131,15336211,North Beacon Hill,5.0,False,136,vQCWx0P7DExHD-KIfA7keA,58.187500,...,2019-03-12 21:55:46.859,5.0,False,8,0,0,"[ ""no alternate route"" ]",None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30532 47.57760)
2,67992286,NoCurbRamp,5951,6443978,North Beacon Hill,5.0,False,138,vQCWx0P7DExHD-KIfA7keA,58.187500,...,2019-03-12 21:55:50.677,5.0,False,2,0,0,"[ ""no alternate route"" ]",None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30527 47.57758)
3,67987353,CurbRamp,5951,6443978,North Beacon Hill,4.0,False,140,vQCWx0P7DExHD-KIfA7keA,140.687500,...,2019-03-12 21:56:08.848,4.0,False,2,0,0,"[ ""narrow"" ]",None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30530 47.57749)
4,67992286,NoCurbRamp,5951,6443978,North Beacon Hill,5.0,False,141,zsvKv7NrQo5DyRZJWnUKlQ,311.312500,...,2019-03-12 21:56:20.754,5.0,False,3,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30527 47.57758)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38581,67949697,NoCurbRamp,4653,6422865,Georgetown,5.0,False,233170,hFVojVcV0UAWHtVa9Nnk7Q,274.250000,...,2023-02-06 01:50:38.739,5.0,False,0,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.32316 47.55050)
38582,67949740,NoCurbRamp,5267,6431368,Georgetown,5.0,False,233171,hFVojVcV0UAWHtVa9Nnk7Q,155.562500,...,2023-02-06 01:50:47.995,5.0,False,1,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.32305 47.55037)
38583,67949717,NoCurbRamp,4652,6422865,Georgetown,5.0,False,233172,hFVojVcV0UAWHtVa9Nnk7Q,155.562500,...,2023-02-06 01:50:58.805,5.0,False,1,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.32295 47.55040)
38584,67947857,CurbRamp,5268,6431368,Georgetown,2.0,False,233173,TWpkUJtmDqHWnV92lykzWw,147.562500,...,2023-02-06 02:01:13.047,2.0,False,1,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.32233 47.55106)


In [5]:
# break geometry into lat and lng
df['lat'] = df.geometry.y
df['lng'] = df.geometry.x

In [6]:
df0 = cluster_label_type_at_index(df,0)[2]

In [7]:
df1 = cluster_label_type_at_index(df,1)[2]

In [8]:
df2 = cluster_label_type_at_index(df,2)[2]

In [9]:
df3 = cluster_label_type_at_index(df,3)[2]

In [10]:
df4 = cluster_label_type_at_index(df,4)[2]

In [11]:
df5 = cluster_label_type_at_index(df,5)[2]

In [12]:
df6 = cluster_label_type_at_index(df,6)[2]

In [13]:
df7 = cluster_label_type_at_index(df,7)[2]

In [14]:
df8 = cluster_label_type_at_index(df,8)[2]

In [15]:
df0["clustered"] = df0.duplicated(subset="cluster_id", keep=False)
df1["clustered"] = df1.duplicated(subset="cluster_id", keep=False)
df2["clustered"] = df2.duplicated(subset="cluster_id", keep=False)
df3["clustered"] = df3.duplicated(subset="cluster_id", keep=False)
df4["clustered"] = df4.duplicated(subset="cluster_id", keep=False)
df5["clustered"] = df5.duplicated(subset="cluster_id", keep=False)
df6["clustered"] = df6.duplicated(subset="cluster_id", keep=False)
df7["clustered"] = df7.duplicated(subset="cluster_id", keep=False)
df8["clustered"] = df8.duplicated(subset="cluster_id", keep=False)

In [16]:
df0

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,...,disagree_count,notsure_count,label_tags,label_description,user_id,geometry,lat,lng,cluster_id,clustered
0,68037009,CurbRamp,19897,263712791,Atlantic,1.0,False,15,jQdLwvjVMqab7AAsMcrVQA,87.004463,...,0,0,[ ],None,e4ff325c-309b-4ea6-8555-8f973e662362,POINT (-122.30888 47.59828),47.598278,-122.308876,5589,True
3,67987353,CurbRamp,5951,6443978,North Beacon Hill,4.0,False,140,vQCWx0P7DExHD-KIfA7keA,140.687500,...,0,0,"[ ""narrow"" ]",None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30530 47.57749),47.577488,-122.305298,1231,False
5,67987353,CurbRamp,5951,6443978,North Beacon Hill,4.0,False,142,zsvKv7NrQo5DyRZJWnUKlQ,251.125000,...,0,1,"[ ""narrow"" ]",None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30530 47.57749),47.577488,-122.305298,1572,False
10,67987345,CurbRamp,5952,6443978,North Beacon Hill,4.0,False,147,b0eOV36e75GdxsTJKxhJzw,144.062500,...,0,1,"[ ""missing tactile warning"", ""not enough landi...",None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30428 47.57747),47.577473,-122.304276,2870,True
18,67987015,CurbRamp,18879,244303612,North Beacon Hill,1.0,False,189,DZtX1-3WTIFwW5UFbYpS-A,223.750000,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30441 47.57845),47.578445,-122.304405,3354,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38558,67947675,CurbRamp,4227,6413364,Georgetown,1.0,False,233132,kHST83zC6EimV-uc7c05AA,5.312500,...,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.31722 47.54874),47.548737,-122.317223,40,True
38572,67947726,CurbRamp,8349,6482719,Georgetown,2.0,False,233160,CQNwcJ3C5fPC3sQ0gJR4NQ,70.250000,...,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.32098 47.54860),47.548603,-122.320976,28,True
38573,67947697,CurbRamp,9578,6503125,Georgetown,1.0,False,233161,CQNwcJ3C5fPC3sQ0gJR4NQ,332.000000,...,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.32124 47.54865),47.548649,-122.321243,22,True
38584,67947857,CurbRamp,5268,6431368,Georgetown,2.0,False,233173,TWpkUJtmDqHWnV92lykzWw,147.562500,...,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.32233 47.55106),47.551060,-122.322334,15,True


In [17]:
#concatenate all dataframes
clusters_q3 = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8])

In [18]:
clusters_q3

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,...,disagree_count,notsure_count,label_tags,label_description,user_id,geometry,lat,lng,cluster_id,clustered
0,68037009,CurbRamp,19897,263712791,Atlantic,1.0,False,15,jQdLwvjVMqab7AAsMcrVQA,87.004463,...,0,0,[ ],None,e4ff325c-309b-4ea6-8555-8f973e662362,POINT (-122.30888 47.59828),47.598278,-122.308876,5589,True
3,67987353,CurbRamp,5951,6443978,North Beacon Hill,4.0,False,140,vQCWx0P7DExHD-KIfA7keA,140.687500,...,0,0,"[ ""narrow"" ]",None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30530 47.57749),47.577488,-122.305298,1231,False
5,67987353,CurbRamp,5951,6443978,North Beacon Hill,4.0,False,142,zsvKv7NrQo5DyRZJWnUKlQ,251.125000,...,0,1,"[ ""narrow"" ]",None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30530 47.57749),47.577488,-122.305298,1572,False
10,67987345,CurbRamp,5952,6443978,North Beacon Hill,4.0,False,147,b0eOV36e75GdxsTJKxhJzw,144.062500,...,0,1,"[ ""missing tactile warning"", ""not enough landi...",None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30428 47.57747),47.577473,-122.304276,2870,True
18,67987015,CurbRamp,18879,244303612,North Beacon Hill,1.0,False,189,DZtX1-3WTIFwW5UFbYpS-A,223.750000,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30441 47.57845),47.578445,-122.304405,3354,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37668,68007730,Signal,27506,648876365,Columbia City,NaN,False,211023,GFsQCc_PMNUAyKm0UxdQ5w,55.026787,...,2,1,"[ ""has button"", ""button waist height"" ]",None,d57a2ee7-6b2c-4ef4-a04d-c8d688f78ca2,POINT (-122.28497 47.55795),47.557945,-122.284973,10,False
37669,68007729,Signal,27506,648876365,Columbia City,NaN,False,211027,GFsQCc_PMNUAyKm0UxdQ5w,99.979912,...,1,0,"[ ""has button"", ""button waist height"" ]",None,d57a2ee7-6b2c-4ef4-a04d-c8d688f78ca2,POINT (-122.28489 47.55780),47.557804,-122.284889,8,False
37672,68007736,Signal,7678,6471331,Columbia City,NaN,False,211032,BfyqKRJWYfRsUcgKDx3Ekg,309.301331,...,1,0,"[ ""has button"", ""button waist height"" ]",None,d57a2ee7-6b2c-4ef4-a04d-c8d688f78ca2,POINT (-122.28485 47.55713),47.557125,-122.284851,6,False
37675,68007735,Signal,7679,6471331,Columbia City,NaN,False,211045,LqW47TLaS5yMPwHcmz_I9A,196.435272,...,2,0,"[ ""has button"", ""button waist height"" ]",None,d57a2ee7-6b2c-4ef4-a04d-c8d688f78ca2,POINT (-122.28445 47.55696),47.556965,-122.284447,4,False


In [19]:
#export to geojson
clusters_q3.to_file("data/clusters_q3.geojson", driver='GeoJSON')